In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import string
import re
import os
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import numpy as np
import pandas as pd

In [14]:
def load_doc(filename):
    #open the file  as read only
    file= open(filename,'r')
    # read all text
    text=file.read()
    #close the file
    file.close()
    return text

In [15]:
text= load_doc('drive/MyDrive/review_polarity/txt_sentoken/pos/cv026_29325.txt')
text

'for those of us who weren\'t yet born when the 1960\'s rock \'n\' rolled around , monterey pop affords an affectionate glimpse of the music that influenced our parents to be hippies . \nfrom otis redding to jimi hendrix , janis joplin to the mamas and the papas , and jefferson airplane to the who , this documentary is jam-packed with contagious energy . \nbut i give fair warning that i will reveal the ending , which does not do the rest of the film the justice it deserves . \nshot in 1969 at an outdoor concert that precluded woodstock , the film defies the stereotype of the general population at the time . \nsure , some have painted their faces and smoke joints , but d . a . \npennebaker ( the war room , moon over broadway ) surprisingly chooses to show a broad spectrum of the audience . \nno matter who is watching , it all comes back to the talented musicians that stir your soul . \nthe excitement starts before the music even begins . \na young girl is cleaning thousands of seats and

In [16]:
# turn a doc into clean tokens
def clean_doc(doc):
  #split into tokens by white space
  tokens= doc.split()
  #prepare regex for char filtering
  re_punc= re.compile('[%s]'% re.escape(string.punctuation))
  #remove punctiuation from each word
  tokens= [re_punc.sub('',w) for w in tokens]
  # remove remaining tokens that are not alphabetic
  tokens=[word for word in tokens if word.isalpha()]
  # filterout stopwords
  stop_words= set(stopwords.words('english'))
  tokens= [w for w in tokens if not w in stop_words]
  # filter out short tokens
  tokens= [word for word in tokens if len(word) > 1]
  return tokens

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
clean= clean_doc(text)
len(clean)

261

In [19]:
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
  #load the doc
  doc=load_doc(filename)
  # clean doc
  tokens= clean_doc(doc)
  #filter by vocab
  tokens=[w for w in tokens if w in vocab]
  return ' '.join(tokens)

In [24]:
vocab= open('drive/MyDrive/datasets/vocab.txt')
vocab=vocab.read().split()

In [25]:
doc_to_line('drive/MyDrive/review_polarity/txt_sentoken/pos/cv026_29325.txt',vocab)

'us werent yet born rock rolled around monterey pop affectionate glimpse music influenced parents hippies otis redding jimi hendrix janis joplin mamas papas jefferson airplane documentary jampacked contagious energy give fair warning reveal ending rest film justice deserves shot outdoor concert film defies stereotype general population time sure painted faces smoke joints war room moon broadway surprisingly chooses show broad spectrum audience matter watching comes back talented musicians stir soul excitement starts music even begins young girl cleaning thousands seats asked interviewer replies feels lucky moments organized craziness john phillips leader mamas papas one concert tries get touch one band tuning member remarks finally decent sound system tell watching first moments show isnt vanity playing music love appreciation street interaction audience performer continues throughout film becomes infectious audience impossible tear eyes away janis joplin belts ballad love ball chain l

In [26]:
from pickle import load
def process_train(directory, vocab):
  document= list()
  for filename in listdir(directory):
    if not filename.startswith('cv9'):
      path= directory+'/'+filename
      doc= load_doc(path)
      tokens= claen_doc(doc, vocab)
      documents.append(tokens)
  return documents

In [27]:
from pickle import load
def process_test(directory, vocab):
  document= list()
  for filename in listdir(directory):
    if filename.startswith('cv9'):
      path= directory+'/'+filename
      doc= load_doc(path)
      tokens= claen_doc(doc, vocab)
      documents.append(tokens)
  return documents

In [28]:
from os import listdir
# load all docs in dictiionary
def process_docs(directory,vocab, is_train):
  documents=list()
  #walk through all files in the folder
  for filename in listdir(directory):
    # skip any review in the test set
    if is_train and filename.startswith('cv9'):
      continue
    if not is_train and filename.startswith('cv9'):
      continue
    # create the full path of the file to open
    path= directory+'/'+filename
    #load the doc
    doc= load_doc(path)
    #claen doc
    tokens= clean_doc(doc)
    #add to list
    documents.append(tokens)
  return documents

In [29]:
lines= process_docs('drive/MyDrive/review_polarity/txt_sentoken/pos',vocab,False)
len(lines)

900

In [30]:
lines= process_docs('drive/MyDrive/review_polarity/txt_sentoken/pos',vocab,True)
len(lines)

900

In [39]:
# loadand clean a dataset
def load_clean_dataset(vocab,is_train):
  #load documents
  neg=process_docs('drive/MyDrive/review_polarity/txt_sentoken/neg', vocab, is_train)
  pos=process_docs('drive/MyDrive/review_polarity/txt_sentoken/pos', vocab, is_train)
  docs=neg+pos

  #prepare labels
  labels=[0 for _ in range(len(neg))]+[1 for _ in range(len(pos))]
  return docs, labels

In [40]:
train,train_labels= load_clean_dataset(vocab,True)
test,test_labels= load_clean_dataset(vocab,False)

In [41]:
len(train),len(train_labels)

(1810, 1810)

In [42]:
len(test),len(test_labels)

(1810, 1810)

In [38]:
from keras.models import Sequential
from keras.layers import Dense

In [47]:
# define the model
def define_model (in_words):
  # define network
  model= Sequential()
  model.add(Dense(50, input_shape=(n_words,),activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #compile network
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  #summarize defined model
  model.summary()
  plot_model(model,to_file='model.png',show_shapes=True)
  return model

In [43]:
# fit a tokenoizer
def create_tokenizer(lines):
  tokenizer= Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [44]:
# create the tokenizer
tokenizer= create_tokenizer(train)

In [45]:
#encode data
x_train= tokenizer.texts_to_matrix(train, mode='binary')

In [50]:
x_train.shape

(1810, 44277)

In [48]:
x_test=tokenizer.texts_to_matrix(test, mode='binary')

In [49]:
x_test.shape

(1810, 44277)

In [52]:
# define network
n_words= x_train.shape[1]
model=define_model(n_words)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2213900   
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2213951 (8.45 MB)
Trainable params: 2213951 (8.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
model.fit(x_train,np.array(train_labels),epochs=10, batch_size=10)

Epoch 1/10
181/181 [==============================] - 20s 107ms/step - loss: 0.4243 - accuracy: 0.7972
Epoch 2/10
181/181 [==============================] - 8s 43ms/step - loss: 0.0285 - accuracy: 0.9956
Epoch 3/10
181/181 [==============================] - 10s 55ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 4/10
181/181 [==============================] - 9s 50ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 5/10
181/181 [==============================] - 9s 49ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 6/10
181/181 [==============================] - 11s 58ms/step - loss: 4.8017e-04 - accuracy: 1.0000
Epoch 7/10
181/181 [==============================] - 8s 44ms/step - loss: 2.5897e-04 - accuracy: 1.0000
Epoch 8/10
181/181 [==============================] - 10s 56ms/step - loss: 1.6424e-04 - accuracy: 1.0000
Epoch 9/10
181/181 [==============================] - 9s 52ms/step - loss: 1.1614e-04 - accuracy: 1.0000
Epoch 10/10
181/181 [==============================] - 10s 53ms/step -